In [19]:
import os
import pandas as pd
import datetime

In [67]:
path = 'files/'
files = os.listdir(path)

In [ ]:
def multiple_csv(path):
    files = os.listdir(path)
    df = pd.DataFrame()
    for i in files:
        data = pd.read_csv(files)
        data.drop(1,axis=0)
        df['Month'] = 
        df.append(data)
    return df